In [1]:
# import packages
import pandas as pd
import os
from dotenv import load_dotenv
import mysql.connector

In [2]:
# Read the CSV file, specifying the latitude and longitude columns as strings
df = pd.read_csv('../data/raw/product_category_name_translation.csv')

In [3]:
df.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [5]:
df.isna().sum()

product_category_name            0
product_category_name_english    0
dtype: int64

In [6]:
# assess missing values
df[df.isna().any(axis=1)]

,product_category_name,product_category_name_english


In [7]:
# check for duplicate primary key
df['product_category_name'].duplicated().sum()

0

In [8]:
# Create table, login and out of MySQL, and load data

# Load environment variables from .env file
load_dotenv()

# Get the password from the environment variable
db_password = os.getenv('DB_PASSWORD')

# Define the function
def create_and_load_table(connection, table_name, columns_def, df):
    cursor = connection.cursor()

    # Create table if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {columns_def}
    )
    """
    cursor.execute(create_table_query)

    # Prepare the SQL query to insert data
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Insert DataFrame values into MySQL table
    for index, row in df.iterrows():
        cursor.execute(insert_query, tuple(row))

    # Commit the transaction
    connection.commit()

    # Close the cursor
    cursor.close()

    
# Get the password from the environment variable
db_password = os.getenv('DB_PASSWORD')

if db_password is None:
    raise ValueError("DB_PASSWORD environment variable is not set")

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password=db_password,
    database='olist_db'
)

# input: enter the the variable name for the desired dataframe to load
table_name = 'cat_translation' #edit this line

# input: define the columns (edit the below)
columns_def = """
product_category_name VARCHAR(100) PRIMARY KEY, 
product_category_name_english VARCHAR(100)
"""

# Call the function
create_and_load_table(connection, table_name, columns_def, df)

# Close the connection
connection.close()